In [1]:
%matplotlib inline

In [2]:
#引入需要的包
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from tqdm import tqdm
np.random.seed(42)

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

Using TensorFlow backend.


In [3]:
data_dir = './Images'
dog_breeds = []
for name in os.listdir(data_dir):
    dog_breeds.append(name.split('-')[1].lower())
dog_breeds = sorted(dog_breeds)
n_classes = len(dog_breeds)
print(n_classes)
dog_breeds[:10]

120


['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle']

In [4]:
class_to_num = dict(zip(dog_breeds, range(n_classes)))

In [5]:
data_size = sum([len(files) for root,dirs,files in os.walk(data_dir)])
print(data_size)

20580


In [6]:
def images_to_array(data_dir, img_size = (224,224,3)):
    '''
    1- 从数据集的目录中读取图像
    2- 将图像大小进行调整，并存放在一个numpy数组里
    3- 读取图像对应的品种信息
    4- 将品种信息转化为One hot编码
    5- 将图像信息和品种信息进行随机打乱
    '''
    X = np.zeros([data_size, img_size[0], img_size[1], img_size[2]], dtype=np.uint8)
    y = np.zeros([data_size,1], dtype=np.uint8)
    #read data and lables.
    cnt_dir = 0
    cnt_file = 0
    for root,dirs,files in os.walk(data_dir):
        for now_file in files:
            img_dir = os.path.join(root, now_file)
            img_pixels = load_img(img_dir, target_size=img_size)
            X[cnt_file] = img_pixels

            image_breed = root.split('-')[1].lower()
            y[cnt_file] = class_to_num[image_breed]
            cnt_file += 1
            print("%3d / %3d , %5d / %5d done!    "%(cnt_dir,n_classes,cnt_file,data_size),end='\r')
        cnt_dir += 1
        print("%3d / %3d , %5d / %5d done!    "%(cnt_dir,n_classes,cnt_file,data_size),end='\r')
        
    print()
    #One hot encoder
    print('To one hot encoding!             ',end='\r')
    y = to_categorical(y)
    print('one hot encoding done!           ',end='\r')
    #shuffle  
    print('shuffling!                       ',end='\r')
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('shuffling done!                  ',end='\r')
    print('To one hot encoding and shuffling done               ')
    print('Ouptut Data Size : ' ,X.shape)
    print('Ouptut Label Size : ' ,y.shape)
    return X, y

In [7]:
#图像大小选择为(331,331,3)
img_size = (331,331,3)
X, y = images_to_array(data_dir, img_size)

121 / 120 , 20580 / 20580 done!    
To one hot encoding and shuffling done               
Ouptut Data Size :  (20580, 331, 331, 3)
Ouptut Label Size :  (20580, 120)


In [8]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=0)
print("train Data Size : ",X_train.shape[0])
print("test Data Size : ",X_test.shape[0])

train Data Size :  14406
test Data Size :  6174


In [9]:
del X,y

In [10]:
def get_features(model_name, data_preprocessor, input_size, data):
    '''
    1- 建立一个特征提取器从数据中提取特征
    2- 返回提取到的特征
    '''
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=128, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [11]:
# 使用 InceptionV3 作为特征提取器
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, X_train)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



14406/14406 [==============================] - 52s 4ms/step
Feature maps shape:  (14406, 2048)


In [12]:
# 使用 Xception 作为特征提取器
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_features(Xception,
                                 xception_preprocessor,
                                 img_size, X_train)

14406/14406 [==============================] - 71s 5ms/step
Feature maps shape:  (14406, 2048)


In [13]:
# 使用 NASNetLarge 作为特征提取器
from keras.applications.nasnet import NASNetLarge, preprocess_input
nasnet_preprocessor = preprocess_input
nasnet_features = get_features(NASNetLarge,
                               nasnet_preprocessor,
                               img_size, X_train)

14406/14406 [==============================] - 192s 13ms/step
Feature maps shape:  (14406, 4032)


In [14]:
# 使用 InceptionResNetV2 作为特征提取器
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X_train)

14406/14406 [==============================] - 99s 7ms/step
Feature maps shape:  (14406, 1536)


In [15]:
# 释放内存空间
del X_train

In [16]:
final_features = np.concatenate([inception_features,
                                 xception_features,
                                 nasnet_features,
                                 inc_resnet_features,], axis=-1)
print('Final feature maps shape', final_features.shape)

Final feature maps shape (14406, 9664)


In [17]:
from keras.callbacks import EarlyStopping
#Prepare call backs
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [18]:
# 构建DNN模型
dnn = keras.models.Sequential([
    InputLayer(final_features.shape[1:]),
    Dropout(0.7),
    Dense(n_classes, activation='softmax')
])

dnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 用提取到的特征训练DNN
h = dnn.fit(final_features, y_train,
            batch_size=512,
            epochs=60,
            validation_split=0.1,
            callbacks=my_callback)

Train on 12965 samples, validate on 1441 samples
Epoch 1/60
12965/12965 [==============================] - 5s 396us/step - loss: 1.3162 - accuracy: 0.7753 - val_loss: 0.1831 - val_accuracy: 0.9403
Epoch 2/60
12965/12965 [==============================] - 1s 52us/step - loss: 0.1698 - accuracy: 0.9440 - val_loss: 0.1643 - val_accuracy: 0.9480
Epoch 3/60
12965/12965 [==============================] - 1s 52us/step - loss: 0.1435 - accuracy: 0.9530 - val_loss: 0.1621 - val_accuracy: 0.9452
Epoch 4/60
12965/12965 [==============================] - 1s 52us/step - loss: 0.1252 - accuracy: 0.9602 - val_loss: 0.1635 - val_accuracy: 0.9507
Epoch 5/60
12965/12965 [==============================] - 1s 53us/step - loss: 0.1159 - accuracy: 0.9603 - val_loss: 0.1600 - val_accuracy: 0.9473
Epoch 6/60
12965/12965 [==============================] - 1s 51us/step - loss: 0.1015 - accuracy: 0.9665 - val_loss: 0.1630 - val_accuracy: 0.9500
Epoch 7/60
12965/12965 [==============================] - 1s 51us/st

In [19]:
inception_features = get_features(InceptionV3, inception_preprocessor, img_size, X_test)

6174/6174 [==============================] - 28s 5ms/step
Feature maps shape:  (6174, 2048)


In [20]:
xception_features = get_features(Xception, xception_preprocessor, img_size, X_test)

6174/6174 [==============================] - 37s 6ms/step
Feature maps shape:  (6174, 2048)


In [21]:
nasnet_features = get_features(NASNetLarge, nasnet_preprocessor, img_size, X_test)

6174/6174 [==============================] - 97s 16ms/step
Feature maps shape:  (6174, 4032)


In [22]:
inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, X_test)

6174/6174 [==============================] - 55s 9ms/step
Feature maps shape:  (6174, 1536)


In [23]:
# 提取测试集的特征
test_features = np.concatenate([inception_features,
                                 xception_features,
                                 nasnet_features,
                                 inc_resnet_features],axis=-1)
print('Final feature maps shape', test_features.shape)

Final feature maps shape (6174, 9664)


In [24]:
# 预测
y_pred = dnn.predict(test_features, batch_size=128)

In [39]:
y_result = y_pred.argmax(axis=1)
y_test_result = y_test.argmax(axis=1)

In [40]:
print(y_result)
print(y_test_result)

[ 85  62  40 ...  39 114  67]
[ 85  62  40 ...  39 114  67]


In [42]:
print((y_result == y_test_result.astype(int)).sum()/y_test_result.shape[0])

0.9442824748947198


In [43]:
print(y_pred)

[[1.2796981e-05 1.3250595e-06 7.1861774e-07 ... 1.1318945e-07
  2.0019799e-07 1.4043926e-06]
 [1.1272825e-06 9.4958580e-07 1.1536797e-06 ... 5.3315927e-07
  4.4875711e-07 8.8954891e-07]
 [1.6093598e-07 8.2706998e-07 1.0283359e-06 ... 2.5320762e-07
  2.4572310e-07 9.4702358e-08]
 ...
 [8.5133320e-07 4.0933867e-07 6.8925914e-07 ... 7.4626388e-07
  1.1052952e-05 6.5475359e-07]
 [9.8053363e-07 8.3240536e-07 1.5329607e-06 ... 8.9162114e-07
  1.3118838e-06 1.4811222e-06]
 [1.6023027e-06 2.2533499e-07 2.5402828e-06 ... 4.4936533e-06
  2.0640696e-06 9.9410147e-07]]
